### <span style="color: blue;">Code reference </span>
#### <span style="color: gray;"> Reference -- Below link was used to come up working Tweeter feed code
##### https://towardsdatascience.com/how-to-build-a-real-time-twitter-analysis-using-big-data-tools-dd2240946e64

# <span style="color: green;"> Required imports to get the feed data from tweeter</span>

In [173]:
from IPython.display import HTML
HTML('''<script>
    code_show_err=false; 
    function code_toggle_err() {
     if (code_show_err){
     $('div.output_stderr').hide();
     } else {
     $('div.output_stderr').show();
     }
     code_show_err = !code_show_err
    } 
    $( document ).ready(code_toggle_err);
    </script>
    To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

In [174]:
#Twitter feed related imports

# For sending GET requests from the API
import requests

# For saving access tokens and for file management when creating and adding to the dataset
import os

# For dealing with json responses we receive from the API
import json

# For displaying the data after
import pandas as pd

# For saving the response data in CSV format
import csv

# For parsing the dates received from twitter in readable formats
from datetime import datetime, timedelta
import dateutil.parser
import unicodedata

import pytz

from pyrfc3339 import generate, parse

# <span style="color: green;"> To get live feed data from tweeter we need to do the following steps:</span>
#### <span style="color: gray;"> 1. Register with twitter for a developer account. https://developer.twitter.com/</span> 
#### <span style="color: gray;"> 2. Create an app within the twitter developer portal</span> 
#### <span style="color: gray;"> 3. Bearer token will be created with app creation</span> 
#### <span style="color: gray;"> 4. This Bearer token will be used to establish session with twitter to receive live feeds</span> 

In [175]:
def auth():
    os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAG7DfwEAAAAAbJLqeNiJMbEMtiCNAc6X3xqUPXE%3DcGSmQEHDbvKKSfbcVNbYrbphIGvDkeZi4wNltyU7mmbGRFvVZq'
    return os.environ['TOKEN']


In [176]:
def create_headers(token):
    headers = {"Authorization":"Bearer {}".format(token)}
    return headers

# <span style="color: green;">There are only certain apis open to developers the list of open apis available for interaction with twitter can be found in </span>
##### https://developer.twitter.com/en/docs/authentication/guides/v2-authentication-mapping

In [177]:
def create_url(key,start_date,end_date,max_results=10):
    search_url = "https://api.twitter.com/2/tweets/search/recent"
    query_params={'query':key,
                  'start_time':start_date,
                  'end_time':end_date,
                  'max_results':max_results,
                  'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                  'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                  'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                  'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                  'next_token': {}}
    return (search_url, query_params)

In [178]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    #print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [179]:
def set_request_metadata():
    #Inputs for the request
    bearer_token = auth()
    headers = create_headers(bearer_token)
    keyword = "fire lang:en"
    st = datetime.utcnow()-timedelta(minutes=15)
    start_time=generate(st.replace(tzinfo=pytz.utc))
    et=datetime.utcnow()-timedelta(seconds=10)
    end_time=generate(et.replace(tzinfo=pytz.utc))
    #start_time = "2022-09-10T19:15:00.000Z"
    #end_time = "2022-09-10T19:30:00.000Z"
    #"2022-08-14T04:26:00.000Z"
    max_results = 15
    return keyword,start_time,end_time,max_results,headers

In [180]:
def get_tweets():
    keyword,start_time,end_time,max_results,headers=set_request_metadata()
    url = create_url(keyword, start_time,end_time, max_results)
    json_response = connect_to_endpoint(url[0], headers, url[1])
    json_str=json.dumps(json_response, indent=4, sort_keys=True)
    return json_str

In [181]:
def get_tweets_df():
    json_tweets=get_tweets()
    tweets_data=[]
    count=0
    loaded_json = json.loads(json_tweets)
    json_texts = loaded_json['data']
    for text in json_texts:
        tweets_data.append(text['text'])
    tweets_data=pd.DataFrame({"tweets":tweets_data})
    return tweets_data